# KPI1 Identification des acteurs les plus présents et les périodes associées
# KPI2 : Evolution de la durée moyenne des films au fil des années
# KPI3 : Comparaison entre les acteurs présents au cinéma et dans les séries
# KPI4 : Age moyen des acteurs.
# KPI5 : Les films les mieux notés et les caractéristiques qu’ils partagent.

## Pour répondre à ces KPI il me faut un filtre sur les tconst de films et et séries qu'il est plus simple de faire sur vscode:
### Après exploration: films => titleType = movie, episode série => titleType = tvEpisode


In [1]:
import pandas as pd

In [26]:
df_t_basics = pd.read_csv("./BD/df_t_basics_clean.csv")

In [23]:
df_basic_name_pbi = pd.read_csv("./BD/df_basic_name_clean.csv")

In [46]:
df_t_ratings = pd.read_csv("./BD/df_t_ratings_clean.csv")

In [29]:
df_t_principals = pd.read_csv("./BD/df_t_principals_clean.csv")

In [37]:
info_acteurs = pd.read_csv("./BD/info_acteurs.csv")

In [ ]:
info_liens = pd.read_csv("./BD/info_liens.csv")

In [4]:
df_tmdb_clean = pd.read_csv("./BD/df_tmdb_clean.csv")

Création DF info_films pour PowerBI

In [34]:
#On filtre les ID pour ne garder que les films et episodes de séries non pornographiques
df_t_basics_pbi = df_t_basics[((df_t_basics['titleType'] == 'movie') | (df_t_basics['titleType'] == 'tvEpisode')) & (df_t_basics['isAdult'] == 0)]

#On ajoute les notes de la table ratings
info_films = pd.merge(df_t_basics_pbi, df_t_ratings, how = 'left', on = 'tconst')

#On sélectionne les colonnes pertinentes
info_films= info_films[['tconst', 'titleType', 'primaryTitle', 'startYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes']]

#Export pour bi
info_films.to_csv("info_films.csv", index = False)

Création DF info_acteurs pour PowerBI

In [ ]:
#On enlèves les colonnes que nous n'utiliseront pas
info_acteurs = df_basic_name_pbi.drop(['knownForTitles', 'deathYear'], axis = 1)

#On ne garde que les acteurs et actrices
info_acteurs = info_acteurs[(info_acteurs[('primaryProfession')].str.contains('actor')) | (info_acteurs[('primaryProfession')].str.contains('actress'))]

#On signale si c'est un acteur ou une actrice avant de droper les autres professions
info_acteurs['profession'] = info_acteurs['primaryProfession'].apply(lambda x : 'actor' if 'actor' in x else 'actress')
info_acteurs = info_acteurs.drop('primaryProfession', axis =1)

#Export
info_acteurs.to_csv("info_acteurs.csv", index = False)

Création DF info_liens pour PowerBI

In [44]:
info_liens =info_liens.drop('category', axis = 1)

In [78]:
#On prend les id de films qui nous intéressent : ils sont déjà triés donc on ne garde que les colonnes id et startYear:
tconst = df_t_basics_pbi[['tconst', 'startYear']]

#On ajoute les id des gens qui ont bossés sur le film:
info_liens = pd.merge(tconst, df_t_principals, how = 'left', on = 'tconst')

#On ne garde que ceux qui sont acteur/actrices:
info_liens = info_liens[(info_liens['category'] == 'actor') | (info_liens['category'] == 'actress')]

#On reprend les acteurs et leur date de naissance:
act_id = info_acteurs[['nconst', 'birthYear']]

#On ajoute les dates de naissance aux id correspondants pour vérifier si toutes les lignes sont rensiegnées:
info_liens= pd.merge(info_liens, act_id, how = 'left', on ='nconst')

#La colonne 'category' ne nous intéresse plus:
info_liens =info_liens.drop('category', axis = 1)

#On créé l'age de l'acteur au moment du film puis on supprime les autres colonnes années:
info_liens['age_act'] = info_liens['startYear'] -info_liens['birthYear']
info_liens = info_liens.drop(['startYear', 'birthYear'], axis = 1)

info_liens = info_liens.reset_index().drop('index', axis = 1)

#Export
info_liens.to_csv("info_liens.csv", index = False)


In [48]:
info_liens.to_csv("info_liens.csv", index = False)

Création DF info_liens_tmdb pour PowerBI

In [51]:
#On veut le même DF que info_liens mais filtré avec seulement les films de la table tmdb_clean:

df_tmdb_clean_pbi = df_tmdb_clean[['ID']]

info_liens_tmdb = pd.merge(info_liens, df_tmdb_clean_pbi, how = 'left', left_on='tconst', right_on = 'ID')

info_liens_tmdb = info_liens_tmdb.dropna().drop('ID', axis = 1)

info_liens_tmdb = info_liens_tmdb.reset_index()

info_liens_tmdb.to_csv("info_liens_tmdb.csv", index = False)


In [52]:
info_liens_tmdb

,index,tconst,nconst,age_act
0,205865,tt0035423,nm0000212,40.0
1,205866,tt0035423,nm0413168,33.0
2,205867,tt0035423,nm0000630,34.0
3,205868,tt0035423,nm0005227,27.0
4,205869,tt0035423,nm0005169,22.0
...,...,...,...,...
278529,29201865,tt9908390,nm1446274,32.0
278530,29209609,tt9913660,nm0217984,45.0
278531,29211616,tt9916190,nm0302466,52.0
278532,29211618,tt9916190,nm2952127,30.0
